# Predict edits to articles in the Tropical Cyclones category

## Goals & Plans

Predict daily edits to pages in jawiki category: Tropical Cyclones (熱帯低気圧)

### Steps

#### ETL to get processed-input dataframes:

- tcyc_page_ids_and_names  (data dump or https://ja.wikipedia.org/wiki/Category:%E7%86%B1%E5%B8%AF%E4%BD%8E%E6%B0%97%E5%9C%A7)
- target_dayseries (edit counts)  (data dump or https://wikimedia.org/api/rest_v1/#/)  (timezone?)
- target_dayseries (pageviews) https://wikimedia.org/api/rest_v1/#/Pageviews_data/get_metrics_pageviews
- jp_cyc (landfall data etc)

#### Quant Analysis

- overall lags
    - Estimate mean/median/mode lags of edits around a tcyc versus landfall
    - T-test (?) of mean lag for each of m/m/m lag to check if it's different from zero
- proportions of edits by time category
    - time categories
        - reactive edits
        - in-season edits
        - off-season edits
    - cross reference time-categories with user-categories
        - weather-specialist editors
        - frequent editors
        - infrequent registered editors
        - IP editors (by location?)
    - cross-reference time-categories with different subcategories
        - tcyc in Japan versus storms in other particular continents
        - tcyc-science articles versus tcyc-storm articles
        - compare edit counts b/o damages, storm strength

#### Predict

- baseline preds by day
    - get residuals from raw values
    - values could be:
        - editcounts
        - residuals
        - reverted-edits
- predict residual pageviews & edits based on landfall, landfall-severity, lagged vars (including self)
    - plot edit-spread around landfall and predictions
- predict non-tcyc edits
    - with similar features
- interesting questions:
    - Are non-tcyc edits "diverted" to tcyc's?
    - Are edits during crises less/more likely to get quickly reverted?
        - Are users whose first edit occurs during a crisis more or less likely to become long-term contributors?

### Details of processed-input dataframes

- Table **tcyc_page_ids_and_names:**
    - Purpose: Help generate the target_dayseries
    - Includes: One row for each page_id / page name combo in wikiproject
    - Primary key (2-col pk): page_id, page name
    - Other columns: none
    - Sorting: By page_id, then by page name

- Table **target_dayseries:**
    - Purpose: hold raw targets, baseline predictions, and deltas for-analysis
    - Includes: One row for every day during period
    - Primary key: edit_day (datetime)
    - Other columns:  
```
editcount_all_raw,        editcount_cat_raw,  
editcount_all_basepreds,     editcount_cat_basepreds,  
delta_all_actual_basepreds,  delta_cat_actual_basepreds,
```

### Sources

- Articles in the Tropical Cyclones category: ([jawiki categories](https://ja.wikipedia.org/wiki/Category:%E7%86%B1%E5%B8%AF%E4%BD%8E%E6%B0%97%E5%9C%A7))  
- Articles to exclude: 
    - People who died in tcyc's ([jawiki categories](https://ja.wikipedia.org/wiki/Category:%E6%B4%9E%E7%88%BA%E4%B8%B8%E4%BA%8B%E6%95%85%E3%81%AE%E7%8A%A0%E7%89%B2%E8%80%85))
    - English-name-articles for names that got assigned to tcyc's: ([jawiki categories](https://ja.wikipedia.org/wiki/Category:%E5%8F%B0%E9%A2%A8%E3%81%AE%E8%8B%B1%E5%90%8D))
- Pageviews:
    - 熱帯低気圧 ([jawiki page](https://ja.wikipedia.org/wiki/%E7%86%B1%E5%B8%AF%E4%BD%8E%E6%B0%97%E5%9C%A7)) ([data source](https://pageviews.toolforge.org/?project=ja.wikipedia.org&platform=all-access&agent=user&redirects=0&range=all-time&pages=%E7%86%B1%E5%B8%AF%E4%BD%8E%E6%B0%97%E5%9C%A7))

## Next-steps

### imports

In [1]:
import pandas as pd, numpy as np, seaborn as sns, mysql.connector as mysql
import os, psycopg2, sqlalchemy, datetime
from pathlib import Path

### read prepped data

In [2]:
damage = pd.read_csv('../data/raw/weather/japan_nii/damage.tsv', sep='\t')
landfall = pd.read_csv('../data/raw/weather/japan_nii/landfall.tsv', sep='\t')

### Import jawiki: {category, categorylinks, page}

Import these tables into a single MySQL database: "jawiki"

```bash
mysql --user=root --password=XXXXXXXX
```

```SQL
CREATE DATABASE jawiki CHARACTER SET utf8 COLLATE utf8_bin;
USE jawiki;
GRANT ALL PRIVILEGES ON jawiki TO bhrdwj@localhost IDENTIFIED BY XXXXXXX;
GRANT ALL PRIVILEGES ON jawiki.* TO bhrdwj@localhost IDENTIFIED BY XXXXXXX;
EXIT
```

```bash
mysql --user=bhrdwj --password=XXXXXXX jawiki < jawiki-20211220-category.sql > output.tab
mysql --user=bhrdwj --password=XXXXXXX jawiki < jawiki-20211220-page.sql > output2.tab
mysql --user=bhrdwj --password=XXXXXXX jawiki < jawiki-20211220-categorylinks.sql > output3.tab

```

### Get sql data

#### turn on sql in bash

```bash
sudo service mysqld start
netstat -lnp | grep mysql
```

In [6]:
mysql_superuser = 'root'
# mysql_su_pass = input(f'Enter the MySQL password for user {mysql_superuser}: ')

#### Initialize a connection and cursor

In [7]:
host='localhost'; user=mysql_superuser; passwd=mysql_su_pass;
cxn = mysql.connect(host=host,user=user,passwd=passwd) # database='latest_categorylinks' # not the first time around
cur = cxn.cursor()

#### Check what's in mysql, pick a database, pick a table

In [8]:
cur.execute('select user();');       print(cur.fetchall())
cur.execute('show databases;');      print(cur.fetchall())

[('root@localhost',)]
[('information_schema',), ('latest_categ',), ('latest_categorylinks',), ('latest_page',), ('mysql',), ('performance_schema',)]


In [9]:
dbname = 'latest_categorylinks'
cur.execute(f'use {dbname};')
cur.execute('select database();');   print(cur.fetchall())
cur.execute('show tables;');         print(cur.fetchall())

[('latest_categorylinks',)]
[('categorylinks',)]


In [10]:
tblname = 'categorylinks'
cur.execute(f'DESCRIBE {tblname};')
schema_categorylinks = pd.DataFrame(cur.fetchall(), columns=['Field','Type','Null','Key','Default','Extra'])
schema_categorylinks;

In [11]:
cur.execute(f'select * from {tblname} limit 10;');   
firstfew = cur.fetchall();   firstfew;

#### Initialize an engine

In [12]:
# Finally, let's instantiate a SQL alchemy engine, so we can pass results sets into pandas and evaluate them here 
connection_str = 'mysql+mysqlconnector://'+user+':'+passwd+'@'+host+'/'+dbname  # removed this after host +':'+dbport
try:
    engine1 = sqlalchemy.create_engine(connection_str)
    conn1 = engine1.connect()
except:
    print('Database connection error - check creds')
metadata = sqlalchemy.MetaData(conn1)
metadata.reflect()
metadata.tables.keys()

dict_keys(['categorylinks'])

In [13]:
sql_eda2 = """select count(*) from categorylinks;"""
print(datetime.datetime.now())
sql_rows = pd.read_sql(sql_eda2,engine1)
print(sql_rows)
print(datetime.datetime.now())


2022-01-04 09:38:35.377515
    count(*)
0  171083519
2022-01-04 09:42:16.266903


## something

https://stackoverflow.com/questions/21782410/finding-subcategories-of-a-wikipedia-category-using-category-and-categorylinks-t

# Failed methods

## scrape page information

Failed because wikipedia's [Extension: CategoryTree](https://www.mediawiki.org/wiki/Extension:CategoryTree) uses AJAX and might require selenium to scrape carefully.

### scrape page identifiers

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
req = requests.get('https://ja.wikipedia.org/wiki/Category:%E7%86%B1%E5%B8%AF%E4%BD%8E%E6%B0%97%E5%9C%A7')
bs = BeautifulSoup(req.content, features='html.parser')

#### get pages

In [ ]:
pages_in_TropicalCyclones_htmlchunk = bs.find('body').find('div',{'id':'mw-pages'}).find('div',{'class':'mw-category'}).find_all('li')

In [ ]:
pages_in_TropicalCyclones = {}
for li in pages_in_TropicalCyclones_htmlchunk:
    pages_in_TropicalCyclones[
            li.find('a').attrs['href']
        ] = li.find('a').attrs['title']

In [ ]:
pages_in_TropicalCyclones

### get categories 

In [ ]:
subcats_in_TropicalCyclones_htmlchunk = bs.find('body').find('div',{'id':'mw-subcategories'}).find_all('div',{'class':'CategoryTreeItem'})

In [ ]:
subcats_in_TropicalCyclones = {}
for li in subcats_in_TropicalCyclones_htmlchunk:
    subcats_in_TropicalCyclones[
            li.find('a').attrs['href']
        ] = li.find('a').attrs['title']

In [ ]:
bs.find('body').find('div',{'id':'mw-subcategories'})

In [ ]:
subcats_in_TropicalCyclones_htmlchunk